In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import pprint

import folium
from folium.plugins import MarkerCluster
import googlemaps
import geopandas as gpd

import chart_studio
chart_studio.tools.set_credentials_file(username='error-_-', api_key='TuIkGS2vNWmn3D5mwRmX')

import requests
import json
import time

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
bus_time_table = pd.read_csv('./datasets/bus_time_table.csv', encoding='cp949', sep=',', parse_dates=['기준_날짜'])
bus_time_table.isna().sum()

기준_날짜            0
행정동_ID           0
버스_승객_수        424
버스_승객_수_00시    424
버스_승객_수_01시    424
버스_승객_수_02시    424
버스_승객_수_03시    424
버스_승객_수_04시    424
버스_승객_수_05시    424
버스_승객_수_06시    424
버스_승객_수_07시    424
버스_승객_수_08시    424
버스_승객_수_09시    424
버스_승객_수_10시    424
버스_승객_수_11시    424
버스_승객_수_12시    424
버스_승객_수_13시    424
버스_승객_수_14시    424
버스_승객_수_15시    424
버스_승객_수_16시    424
버스_승객_수_17시    424
버스_승객_수_18시    424
버스_승객_수_19시    424
버스_승객_수_20시    424
버스_승객_수_21시    424
버스_승객_수_22시    424
버스_승객_수_23시    424
dtype: int64

In [3]:
# 결측치 2022-12-18 삭제
del_idx = bus_time_table[bus_time_table.isna().values == True].index
bus_time_table = bus_time_table.drop(del_idx, axis=0)
bus_time_table.isna().sum()[bus_time_table.isna().sum()>0]

Series([], dtype: int64)

In [4]:
bus_time_table.shape

(173416, 27)

In [5]:
# 승객수 시간별 총합이 버스 승객수가 맞는지 확인
bus_time_table[bus_time_table[['버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']].sum(axis=1) != bus_time_table['버스_승객_수']]

,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,...,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시


#######################

In [6]:
# 컬럼명 변경
bus = pd.DataFrame()
bus['busdate'] = bus_time_table['기준_날짜']
bus['yy'] = bus_time_table['기준_날짜'].dt.year
bus['mm'] = bus_time_table['기준_날짜'].dt.month
bus['dd'] = bus_time_table['기준_날짜'].dt.day
bus['dong_id'] = bus_time_table['행정동_ID']
bus['guest_cnt'] = bus_time_table['버스_승객_수']
bus[['hh00', 'hh01',
       'hh02', 'hh03', 'hh04', 'hh05',
       'hh06', 'hh07', 'hh08', 'hh09',
       'hh10', 'hh11', 'hh12', 'hh13',
       'hh14', 'hh15', 'hh16', 'hh17',
       'hh18', 'hh19', 'hh20', 'hh21',
       'hh22', 'hh23']] = bus_time_table[['버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']].astype('int')

bus = bus.sort_values(['busdate','yy','mm','dd'])
bus

,busdate,yy,mm,dd,dong_id,guest_cnt,hh00,hh01,hh02,hh03,...,hh14,hh15,hh16,hh17,hh18,hh19,hh20,hh21,hh22,hh23
173416,2022-01-01,2022,1,1,1124082,4441.0,10,18,7,3,...,335,362,371,321,210,193,179,272,181,33
173417,2022-01-01,2022,1,1,1124081,4707.0,17,33,21,1,...,315,389,373,367,353,263,257,328,205,42
173418,2022-01-01,2022,1,1,1106091,3076.0,3,0,0,0,...,231,266,241,220,142,141,139,176,89,42
173419,2022-01-01,2022,1,1,1111079,4418.0,32,13,7,10,...,292,300,321,322,317,222,252,364,238,80
173420,2022-01-01,2022,1,1,1121069,11695.0,48,6,7,7,...,701,715,844,892,848,811,887,1445,789,336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,2023-02-15,2023,2,15,1110061,5170.0,13,0,0,0,...,267,227,261,308,333,227,149,129,76,56
420,2023-02-15,2023,2,15,1110060,7534.0,21,4,1,1,...,362,321,326,417,503,334,191,202,150,72
421,2023-02-15,2023,2,15,1110056,4684.0,18,0,2,0,...,272,271,248,221,280,193,117,96,66,34
422,2023-02-15,2023,2,15,1115069,6300.0,18,0,0,0,...,501,409,355,425,569,310,319,324,322,116


컬럼명      wd
공휴일+주말  1 
평일        0

In [7]:
공휴일22_23 = ['2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2022-10-10', '2022-12-25', '2023-01-01', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24']
bus['busdate'].isin(공휴일22_23)

173416     True
173417     True
173418     True
173419     True
173420     True
          ...  
419       False
420       False
421       False
422       False
423       False
Name: busdate, Length: 173416, dtype: bool

In [8]:
# bus = pd.read_csv('./datasets/bus_mycol.csv', parse_dates=['busdate'])
h_arr =  bus[['hh00', 'hh01', 'hh02', 'hh03', 'hh04', 'hh05', 'hh06', 'hh07', 'hh08', 'hh09', 'hh10', 'hh11', 'hh12', 'hh13', 'hh14', 'hh15', 'hh16', 'hh17', 'hh18', 'hh19', 'hh20', 'hh21', 'hh22', 'hh23']].values.reshape(-1, order='F')
hh = np.array([[i]*bus.shape[0] for i in range(24)]).reshape(-1)
b = bus[['busdate','yy','mm','dd','dong_id','guest_cnt']]
b3 = pd.concat([pd.concat([b,b]),b])
b6 = pd.concat([b3,b3])
b12 = pd.concat([b6,b6])
b24 = pd.concat([b12,b12])
b24['ww'] = b24['busdate'].dt.dayofweek
b24['hh'] = hh
b24['g_cnt'] = h_arr
b24['guest_cnt'] = b24['guest_cnt'].astype('int')
wd22_23 = ['2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2022-10-10', '2022-12-25', '2023-01-01', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24']
b24['wd'] = np.where((b24['busdate'].isin(wd22_23))|(b24['ww']==5)|(b24['ww']==6),1,0)
b24.sort_values(['busdate', 'yy', 'mm', 'dd', 'dong_id'])
b24.shape

(4161984, 10)

In [9]:
b24.set_index(['yy','mm','dd','hh'])

busdate  dong_id  guest_cnt  ww  g_cnt  wd
yy   mm dd hh                                              
2022 1  1  0  2022-01-01  1124082       4441   5     10   1
           0  2022-01-01  1124081       4707   5     17   1
           0  2022-01-01  1106091       3076   5      3   1
           0  2022-01-01  1111079       4418   5     32   1
           0  2022-01-01  1121069      11695   5     48   1
...                  ...      ...        ...  ..    ...  ..
2023 2  15 23 2023-02-15  1110061       5170   2     56   0
           23 2023-02-15  1110060       7534   2     72   0
           23 2023-02-15  1110056       4684   2     34   0
           23 2023-02-15  1115069       6300   2    116   0
           23 2023-02-15  1115065       3592   2     46   0

[4161984 rows x 6 columns]

In [10]:
b24[b24['busdate']=='2022-01-01']['dong_id'].unique()

array([1124082, 1124081, 1106091, 1111079, 1121069, 1114076, 1113071,
       1104067, 1124065, 1121079, 1119054, 1115066, 1123053, 1122057,
       1121073, 1121054, 1120054, 1119062, 1118052, 1117055, 1116063,
       1110065, 1120052, 1115058, 1110055, 1109063, 1108071, 1107071,
       1107054, 1105067, 1104073, 1104059, 1101056, 1113076, 1113062,
       1112072, 1111059, 1103058, 1125074, 1125055, 1124064, 1123080,
       1101068, 1120071, 1119075, 1119063, 1118057, 1117065, 1116069,
       1116051, 1110057, 1102052, 1115062, 1114074, 1109072, 1108083,
       1108063, 1107065, 1106087, 1106073, 1105057, 1101061, 1114066,
       1114060, 1113066, 1112065, 1111077, 1103072, 1103052, 1125070,
       1125053, 1124053, 1101071, 1123071, 1125071, 1125065, 1125056,
       1125051, 1124077, 1124069, 1124063, 1124059, 1124055, 1124051,
       1123065, 1123051, 1123076, 1123068, 1122067, 1122063, 1122059,
       1122054, 1121083, 1121071, 1121065, 1121061, 1120073, 1120069,
       1120065, 1120

In [11]:
b24['guest_cnt'].values.reshape(-1,424)

array([[ 4441,  4707,  3076, ...,  2377,  2229,  1353],
       [ 4735,  5555,  3251, ..., 29567,  7493,  3436],
       [11434,  9600,  7287, ...,  4077,  5456,  3139],
       ...,
       [14253, 10542,  7586, ...,  4510,  5781,  3359],
       [14898, 11110,  7854, ...,  4542,  6045,  3522],
       [15015, 11184,  8005, ...,  4684,  6300,  3592]])

In [12]:
b24.pivot(index=['yy', 'mm', 'dd', 'hh'], columns='dong_id', values = 'guest_cnt')

dong_id        1101053  1101054  1101055  1101056  1101057  1101058  1101060  \
yy   mm dd hh                                                                  
2022 1  1  0      9204     1748     3508     4202      926     3960      709   
           1      9204     1748     3508     4202      926     3960      709   
           2      9204     1748     3508     4202      926     3960      709   
           3      9204     1748     3508     4202      926     3960      709   
           4      9204     1748     3508     4202      926     3960      709   
...                ...      ...      ...      ...      ...      ...      ...   
2023 2  15 19    28956     4521     8227     8780     2012    10620     2033   
           20    28956     4521     8227     8780     2012    10620     2033   
           21    28956     4521     8227     8780     2012    10620     2033   
           22    28956     4521     8227     8780     2012    10620     2033   
           23    28956     4521     8227     8780     2012    10620     2033   

dong_id        1101061  1101063  1101064  ...  1125061  1125063  1125065  \
yy   mm dd hh                             ...                              
2022 1  1  0     24600     3968     5063  ...     1839     2585     1966   
           1     24600     3968     5063  ...     1839     2585     1966   
           2     24600     3968     5063  ...     1839     2585     1966   
           3     24600     3968     5063  ...     1839     2585     1966   
           4     24600     3968     5063  ...     1839     2585     1966   
...                ...      ...      ...  ...      ...      ...      ...   
2023 2  15 19    67196    13173    15698  ...     3724     4658     5947   
           20    67196    13173    15698  ...     3724     4658     5947   
           21    67196    13173    15698  ...     3724     4658     5947   
           22    67196    13173    15698  ...     3724     4658     5947   
           23    67196    13173    15698  ...     3724     4658     5947   

dong_id        1125066  1125067  1125070  1125071  1125072  1125073  1125074  
yy   mm dd hh                                                                 
2022 1  1  0      3211     2017      657     2252     1513     5373     4381  
           1      3211     2017      657     2252     1513     5373     4381  
           2      3211     2017      657     2252     1513     5373     4381  
           3      3211     2017      657     2252     1513     5373     4381  
           4      3211     2017      657     2252     1513     5373     4381  
...                ...      ...      ...      ...      ...      ...      ...  
2023 2  15 19     6574     5003     1720     7638     3068    11116    10276  
           20     6574     5003     1720     7638     3068    11116    10276  
           21     6574     5003     1720     7638     3068    11116    10276  
           22     6574     5003     1720     7638     3068    11116    10276  
           23     6574     5003     1720     7638     3068    11116    10276  

[9816 rows x 424 columns]

In [13]:
pd.pivot_table(b24, index=['yy', 'mm', 'dd', 'hh'], columns='dong_id', values = 'guest_cnt')

dong_id        1101053  1101054  1101055  1101056  1101057  1101058  1101060  \
yy   mm dd hh                                                                  
2022 1  1  0      9204     1748     3508     4202      926     3960      709   
           1      9204     1748     3508     4202      926     3960      709   
           2      9204     1748     3508     4202      926     3960      709   
           3      9204     1748     3508     4202      926     3960      709   
           4      9204     1748     3508     4202      926     3960      709   
...                ...      ...      ...      ...      ...      ...      ...   
2023 2  15 19    28956     4521     8227     8780     2012    10620     2033   
           20    28956     4521     8227     8780     2012    10620     2033   
           21    28956     4521     8227     8780     2012    10620     2033   
           22    28956     4521     8227     8780     2012    10620     2033   
           23    28956     4521     8227     8780     2012    10620     2033   

dong_id        1101061  1101063  1101064  ...  1125061  1125063  1125065  \
yy   mm dd hh                             ...                              
2022 1  1  0     24600     3968     5063  ...     1839     2585     1966   
           1     24600     3968     5063  ...     1839     2585     1966   
           2     24600     3968     5063  ...     1839     2585     1966   
           3     24600     3968     5063  ...     1839     2585     1966   
           4     24600     3968     5063  ...     1839     2585     1966   
...                ...      ...      ...  ...      ...      ...      ...   
2023 2  15 19    67196    13173    15698  ...     3724     4658     5947   
           20    67196    13173    15698  ...     3724     4658     5947   
           21    67196    13173    15698  ...     3724     4658     5947   
           22    67196    13173    15698  ...     3724     4658     5947   
           23    67196    13173    15698  ...     3724     4658     5947   

dong_id        1125066  1125067  1125070  1125071  1125072  1125073  1125074  
yy   mm dd hh                                                                 
2022 1  1  0      3211     2017      657     2252     1513     5373     4381  
           1      3211     2017      657     2252     1513     5373     4381  
           2      3211     2017      657     2252     1513     5373     4381  
           3      3211     2017      657     2252     1513     5373     4381  
           4      3211     2017      657     2252     1513     5373     4381  
...                ...      ...      ...      ...      ...      ...      ...  
2023 2  15 19     6574     5003     1720     7638     3068    11116    10276  
           20     6574     5003     1720     7638     3068    11116    10276  
           21     6574     5003     1720     7638     3068    11116    10276  
           22     6574     5003     1720     7638     3068    11116    10276  
           23     6574     5003     1720     7638     3068    11116    10276  

[9816 rows x 424 columns]

In [14]:
b24[(b24['dong_id']==1115065)]

,busdate,yy,mm,dd,dong_id,guest_cnt,ww,hh,g_cnt,wd
173839,2022-01-01,2022,1,1,1115065,1353,5,0,5,1
173350,2022-01-02,2022,1,2,1115065,1494,6,0,2,1
172991,2022-01-03,2022,1,3,1115065,3139,0,0,4,0
172567,2022-01-04,2022,1,4,1115065,3247,1,0,6,0
172143,2022-01-05,2022,1,5,1115065,3247,2,0,6,0
...,...,...,...,...,...,...,...,...,...,...
2119,2023-02-11,2023,2,11,1115065,2516,5,23,29,1
1695,2023-02-12,2023,2,12,1115065,1778,6,23,18,1
1271,2023-02-13,2023,2,13,1115065,3359,0,23,39,0
847,2023-02-14,2023,2,14,1115065,3522,1,23,44,0


In [15]:
b24.columns

Index(['busdate', 'yy', 'mm', 'dd', 'dong_id', 'guest_cnt', 'ww', 'hh',
       'g_cnt', 'wd'],
      dtype='object')

In [16]:
bus[bus['dong_id']==1101053]

,busdate,yy,mm,dd,dong_id,guest_cnt,hh00,hh01,hh02,hh03,...,hh14,hh15,hh16,hh17,hh18,hh19,hh20,hh21,hh22,hh23
173709,2022-01-01,2022,1,1,1101053,9204.0,32,5,1,2,...,652,786,814,930,768,483,584,708,308,109
173220,2022-01-02,2022,1,2,1101053,10137.0,38,6,5,0,...,829,937,951,965,819,610,654,650,298,120
172861,2022-01-03,2022,1,3,1101053,22099.0,83,25,5,3,...,1086,1296,1420,2115,3314,1685,1349,1531,629,220
172437,2022-01-04,2022,1,4,1101053,23359.0,80,26,6,1,...,1095,1442,1439,2195,3290,1709,1432,1876,637,224
172013,2022-01-05,2022,1,5,1101053,23359.0,80,26,6,1,...,1095,1442,1439,2195,3290,1709,1432,1876,637,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,2023-02-11,2023,2,11,1101053,18636.0,172,29,34,4,...,1333,1600,1687,1690,1338,1054,1118,1167,800,500
1565,2023-02-12,2023,2,12,1101053,15129.0,75,17,13,6,...,1277,1348,1548,1404,1344,984,898,848,548,260
1141,2023-02-13,2023,2,13,1101053,26525.0,137,55,20,12,...,1293,1672,1771,2593,3856,2023,1518,1529,1105,518
717,2023-02-14,2023,2,14,1101053,28239.0,161,61,19,6,...,1381,1697,1884,2767,3940,2067,1518,1690,1433,615


In [17]:
bus_junggu = bus[(bus['dong_id']//100==11020)]
bus_junggu

,busdate,yy,mm,dd,dong_id,guest_cnt,hh00,hh01,hh02,hh03,...,hh14,hh15,hh16,hh17,hh18,hh19,hh20,hh21,hh22,hh23
173467,2022-01-01,2022,1,1,1102052,10768.0,56,16,4,2,...,691,894,916,1003,1192,847,884,896,481,134
173571,2022-01-01,2022,1,1,1102073,534.0,4,1,0,0,...,37,34,44,51,28,30,19,32,21,9
173572,2022-01-01,2022,1,1,1102068,1613.0,3,1,0,0,...,133,100,131,113,120,90,83,79,62,33
173573,2022-01-01,2022,1,1,1102059,5557.0,35,32,7,32,...,433,457,494,529,437,418,358,400,232,100
173574,2022-01-01,2022,1,1,1102054,12329.0,138,21,7,7,...,932,1070,1071,1034,869,746,738,907,558,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,2023-02-15,2023,2,15,1102065,6033.0,30,5,2,7,...,319,357,356,401,626,499,344,362,379,197
339,2023-02-15,2023,2,15,1102060,4175.0,61,18,5,10,...,259,307,291,385,460,323,234,273,259,150
367,2023-02-15,2023,2,15,1102058,3973.0,11,6,2,1,...,219,270,405,390,465,240,194,257,175,47
368,2023-02-15,2023,2,15,1102057,4117.0,21,14,8,10,...,262,266,268,301,431,274,203,184,147,87


In [18]:
sns.barplot(data=bus_junggu[bus_junggu['yy']==2022], x='mm', y='g_cnt', hue='wd')
plt.show()

ValueError: Could not interpret input 'g_cnt'

In [ ]:
sns.barplot(data=bus[bus['yy']==2022], x='mm', y='g_cnt', hue='wd')
plt.show()

In [ ]:
# 중구 22년 3월만
bus_junggu_3month = bus[(bus['dong_id']//100==11020)&(bus['mm']==3)].sort_values(['busdate'])
bus_junggu_3month.info()

In [ ]:
# 노원구 22년 3월만
bus_nowon_3month = bus[(bus['dong_id']//100==11110)&(bus['mm']==3)].sort_values(['busdate'])
bus_nowon_3month.info()

In [ ]:
# 강남구 22년 3월만
bus_gangnam_3month = bus[(bus['dong_id']//100==11230)&(bus['mm']==3)].sort_values(['busdate'])
bus_gangnam_3month.info()

In [ ]:
# 송파구 22년 3월만
bus_songpa_3month = bus[(bus['dong_id']//100==11240)&(bus['mm']==3)].sort_values(['busdate'])
bus_songpa_3month.info()

In [ ]:
# 서초구 22년 3월만
bus_seocho_3month = bus[(bus['dong_id']//100==11220)&(bus['mm']==3)].sort_values(['busdate'])
bus_seocho_3month.info()

In [ ]:
# 종로구 22년 3월만
bus_jongro_3month = bus[(bus['dong_id']//100==11010)&(bus['mm']==3)].sort_values(['busdate'])
bus_jongro_3month.info()

In [ ]:
주말 = bus_junggu_3month[(bus_junggu_3month['ww']==5)|(bus_junggu_3month['ww']==6)|(bus_junggu_3month['dd']==1)]
sns.barplot(data=주말, x='hh', y='g_cnt')
plt.show()

In [ ]:
주중 = bus_junggu_3month[(bus_junggu_3month['ww']!=5)&(bus_junggu_3month['ww']!=6)&(bus_junggu_3month['dd']!=1)]
sns.barplot(data=bus, x='hh', y='g_cnt')
plt.show()

In [ ]:
plt.figure(figsize=(14,10))

plt.subplot(1,2,1)
sns.barplot(data=bus_3month, x='hh', y='g_cnt', hue='wd')
plt.ylim([0, 1200])

plt.subplot(1,2,2)
sns.barplot(data=bus_jongro_3month, x='hh', y='g_cnt', hue='wd')
plt.ylim([0, 1200])


plt.show()

In [ ]:
# plt.figure(figsize=(14,14))
sns.barplot(data=bus_nowon_3month, x='hh', y='g_cnt', hue='wd')
plt.show()

In [ ]:
# plt.figure(figsize=(14,14))
sns.barplot(data=bus, x='hh', y='g_cnt', hue='wd')
plt.show()

In [ ]:
sns.barplot(data=bus_3month, x='hh', y='g_cnt')
plt.show()

In [ ]:
sns.barplot(data=bus_nowon_3month, x='hh', y='g_cnt')
plt.show()

In [ ]:
# 서울geojson을 중구geojson으로 잘라서 저장

# state_geo = gpd.read_file('./datasets/seoul_geo_dong.geojson')
# junggu_geo = state_geo[ state_geo['adm_cd'].astype('int')//100 == 11020 ]

# with open('C:\\AI\\pythonProject\\venv\\lecture\\semipjt\\datasets\\junggu_geo.geojson', 'w') as f:
#     f.write(junggu_geo.to_json(drop_id=True))

In [ ]:
# state_geo = gpd.read_file('./datasets/seoul_geo_dong.geojson')
# junggu_geo = state_geo[ state_geo['adm_cd'].astype('int')//100 == 11020 ]
# junggu_geo

# here = [37.560914, 126.990202]
# m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

# folium.GeoJson(junggu_geo).add_to(m)
# m

In [ ]:
# state_geo = gpd.read_file('./datasets/seoul_geo_sigugun.json')
# seoul_ingu = pd.read_csv("./datasets/주민등록인구_20230222161234.csv", sep='\t', encoding='cp949')

# here = [37.560914, 126.990202]
# m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

# m.choropleth(
#     geo_data=state_geo,
#     data=seoul_ingu,
#     columns=['동별', '총계'],
#     key_on='feature.properties.name')
# m

In [ ]:
junggu_geo = gpd.read_file('./datasets/junggu_geo.geojson')
bus_3month_dong_tot = pd.read_csv('./datasets/bus_3month_dong_tot.csv')

here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo).add_to(m)

m.choropleth(
    geo_data=junggu_geo,
    data=bus_3month_dong_tot,
    columns=['dong_id', 'guest_cnt'],
    key_on='feature.properties.adm_cd')
m

# 인구데이터 불러오기

# 중구 3월 주중

In [ ]:
junggu_geo = gpd.read_file('./datasets/junggu_geo.geojson')
junggu_3mm = pd.read_csv('./datasets/버스중구3월.csv')
junggu_3mm_dong_cnt_wd = junggu_3mm[junggu_3mm['wd']==0].groupby('dong_id')[['g_cnt']].mean().reset_index()

here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo).add_to(m)

m.choropleth(
    geo_data=junggu_geo,
    data=junggu_3mm_dong_cnt_wd,
    columns=['dong_id', 'g_cnt'],
    key_on='feature.properties.adm_cd')
m

# 중구 3월 주말

In [ ]:
junggu_geo = gpd.read_file('./datasets/junggu_geo.geojson')
junggu_3mm = pd.read_csv('./datasets/버스중구3월.csv')
junggu_3mm_dong_cnt_we = junggu_3mm[junggu_3mm['wd']==1].groupby('dong_id')[['g_cnt']].mean().reset_index()

here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo).add_to(m)

m.choropleth(
    geo_data=junggu_geo,
    data=junggu_3mm_dong_cnt_we,
    columns=['dong_id', 'g_cnt'],
    key_on='feature.properties.adm_cd')
m

In [ ]:
# 인구당승객

junggu_geo = gpd.read_file('./datasets/junggu_geo.geojson')
서울인구 = pd.read_csv('./datasets/gu_dong_population_2022_03.csv', encoding='cp949')
중구인구 = 서울인구[서울인구['구별']=='중구']
중구인구['코드'] = ['1102052','1102054','1102055','1102057','1102058','1102059','1102060','1102065','1102067','1102068','1102069','1102070','1102071','1102072','1102073']
중구인구['소계'] = 중구인구['소계'].str.replace(',', '').astype('int')
중구인구['코드'] = 중구인구['코드'].astype('int')
중구3월 = pd.read_csv('./datasets/버스중구3월.csv')
중구3월동별총합 = 중구3월.groupby('dong_id')[['g_cnt']].sum().reset_index()
중구3월동별총합['코드'] = 중구3월동별총합['dong_id']
중구인구_승객 = pd.merge(중구인구, 중구3월동별총합, how='inner', on='코드')
중구인구_승객['인구당승객'] = 중구인구_승객['g_cnt']/중구인구_승객['소계']
중구동별인구당승객 = 중구인구_승객[['dong_id','인구당승객']]

here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo).add_to(m)

m.choropleth(
    geo_data=junggu_geo,
    data=중구동별인구당승객,
    columns=['dong_id', '인구당승객'],
    key_on='feature.properties.adm_cd')
m